### Tools

Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:

1. A schema, including the name of the tool, a description, and/or argument definitions (oftten a JSON schema)
2. A function or coroutine to execute.

In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They\'re birds known for their ability to mimic human speech. I remember that some parrots like African Greys, Macaws, and Budgerigars (Budgies) are particularly good at it. But why do they do that? Is it just to copy sounds, or is there more to it?\n\nFirst, I think about their natural environment. In the wild, parrots live in social groups. Maybe they imitate sounds to communicate with each other. I\'ve heard that some parrots use mimicry as a way to bond with their flock. So maybe in captivity, they do the same with humans. But how does that work? Do they see humans as part of their flock?\n\nThen there\'s the aspect of learning. Parrots have a part of their brain called the song system, which is involved in vocal learning. This is similar to how humans learn to speak. Maybe they have a strong neural mechanism for learning sounds. Also, their syrinx (the b

In [3]:
from langchain.tools import tool

@tool
def get_weather(location:str) -> str:
    """ Get the weather at a location """

    return f"It's sunny in {location}"

model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke(["What is the weather like in Boston?"])

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which in this case is Boston. I\'ll call the function with "Boston" as the location argument. Make sure the JSON is correctly formatted with the name and arguments.\n', 'tool_calls': [{'id': '9mc639jgm', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 154, 'total_tokens': 245, 'completion_time': 0.141055957, 'completion_tokens_details': {'reasoning_tokens': 67}, 'prompt_time': 0.006576497, 'prompt_tokens_details': None, 'queue_time': 0.159548823, 'total_time': 0.147632454}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'},

In [4]:
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call["name"]}")
    print(f"Args: {tool_call["args"]}")


### Tool Execution Loops

In [ ]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)


# Step 2: Execute tools and collect result
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)

The current weather in Boston is sunny.
